In [1]:
from google.colab import drive
import os
drive.mount("/content/drive")

PROJECT_DIRECTORY = "drive/MyDrive/DeepL"
os.chdir(PROJECT_DIRECTORY)

!ls
!pip install -r requirements.txt

Mounted at /content/drive
data  log_file_test.log  papers  requirements.txt  src	weights
     |████████████████████████████████| 15.7 MB 6.2 MB/s 
     |████████████████████████████████| 388 kB 47.0 MB/s 
     |████████████████████████████████| 60.3 MB 63 kB/s 
     |████████████████████████████████| 11.3 MB 52.2 MB/s 
     |████████████████████████████████| 11.2 MB 44.7 MB/s 
     |████████████████████████████████| 45 kB 832 kB/s 
     |████████████████████████████████| 3.3 MB 50.0 MB/s 
     |████████████████████████████████| 26.9 MB 1.3 MB/s 
     |████████████████████████████████| 890 kB 38.7 MB/s 
     |████████████████████████████████| 3.1 MB 52.7 MB/s 
  Created wheel for moviepy: filename=moviepy-1.0.3-py3-none-any.whl size=110744 sha256=e82822b85cd7dd5bdf157ea6b96089d017b46c1f99ce41f06a8821051142aefe
  Stored in directory: /root/.cache/pip/wheels/56/dc/2b/9cd600d483c04af3353d66623056fc03faed76b7518faae4df
  Created wheel for proglog: filename=proglog-0.1.9-py3-none-any.whl siz

In [8]:
import os
import numpy as np
from tqdm import tqdm
from torch.utils.data import DataLoader

from src.models.cartoon_gan import CartoonGan
from src import config
from src.dataset.dataset_pictures import PicturesDataset
from src.preprocessing.preprocessor import Preprocessor

preprocessor = Preprocessor(size=256)

test_pictures_dataset = PicturesDataset(
    train=False,
    transform=preprocessor.picture_preprocessor(),
    size=50
)

cartoon_gan = CartoonGan(
    nb_channels_1_h_l_disc=32,
    nb_channels_1_h_l_gen=64,
    nb_channels_cartoon=3,
    nb_channels_picture=3,
    nb_resnet_blocks=8,
)

cartoon_gan.load_model(
    generator_path=os.path.join(config.WEIGHTS_FOLDER, "pretrained", "pretrained_gen_11.pkl"),
    discriminator_path=os.path.join(config.WEIGHTS_FOLDER, "pretrained", "pretrained_disc_11.pkl")
)

test_pictures_loader = DataLoader(
    dataset=test_pictures_dataset,
    batch_size=8,
    shuffle=False,
    # drop last incomplete batch
    drop_last=False,
    num_workers=2
)

### Save images

In [3]:
PRETRAINED_FOLDER = os.path.join("data", "results", "pretrained")

In [9]:

cartoons = cartoon_gan.cartoonize_dataset(test_pictures_loader)

for i, cartoon in enumerate(cartoons):
    np.save(os.path.join(PRETRAINED_FOLDER, f"pretrained_{i}.npy"), cartoon)


100%|██████████| 7/7 [01:50<00:00, 15.83s/it]


### Display images

In [10]:
import matplotlib.pyplot as plt
from PIL import Image
from torchvision  import transforms

!pip install matplotlib==3.1.3

transform = transforms.Compose([
  transforms.ToTensor(),
  transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])

for cartoon_path in os.listdir(PRETRAINED_FOLDER):
    with open(os.path.join(PRETRAINED_FOLDER, cartoon_path), 'rb') as cartoon_file:
      cartoon = np.load(cartoon_file)
      cartoon = (np.stack([cartoon[0], cartoon[1], cartoon[2]], axis=2) + 1) / 2
      print(np.max(cartoon))
      print(np.min(cartoon))
      plt.imshow(cartoon)
      plt.show()


Output hidden; open in https://colab.research.google.com to view.